In [1]:
import os
import sagemaker

if not os.environ.get('MODERN_BERT_ENDPOINT'):
    os.environ["MODERN_BERT_ENDPOINT"] = sagemaker.utils.name_from_base("modernbert-base-std")
    endpoint_name = os.environ["MODERN_BERT_ENDPOINT"]
os.environ["MODERN_BERT_MODEL_ID"] = "answerdotai/ModernBERT-base"
os.environ["MODERN_BERT_INSTANCE"] = "ml.g5.xlarge"
os.environ["AWS_REGION"] = "us-east-1"
os.environ["SAGEMAKER_EXECUTION_ROLE"] = "arn:aws:iam::371087393859:role/defaultrole"
print(endpoint_name)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/lbrenap/.config/sagemaker/config.yaml
modernbert-base-std-2025-12-03-22-38-26-659


In [2]:
from annotations_classification.main import run_linear_svc_classification
from pathlib import Path
from annotations_classification.gpt5_classifier import GPT5Config
annotation_dir = Path("/home/lbrenap/Documents/projects/detecting_arguments/annotations/gregory")

print(annotation_dir)
results = run_linear_svc_classification(
    annotation_dir,
    use_modern_bert=True,
    gpt5=True,
    gpt5_config=GPT5Config(model="gpt-5-mini"),
    test_mode=False,
    combine_analysis_conclusion=False,
    gpt5_checkpoint_path="old_checkpoints/gpt5_mini_predictions.jsonl",
    filter_implicit_conclusions=True,
)

/home/lbrenap/miniconda3/envs/ai_agents/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/lbrenap/Documents/projects/detecting_arguments/annotations/gregory

Embedding: tfidf
                    precision    recall  f1-score   support

          Analysis       0.71      0.69      0.70       134
  Background Facts       0.65      0.75      0.70        40
        Conclusion       0.44      0.15      0.23        26
Procedural History       0.62      0.71      0.66        34
              Rule       0.74      0.82      0.78        99

          accuracy                           0.69       333
         macro avg       0.63      0.62      0.61       333
      weighted avg       0.68      0.69      0.68       333



No sentence-transformers model found with name nlpaueb/legal-bert-small-uncased. Creating a new one with mean pooling.



Embedding: sbert
                    precision    recall  f1-score   support

          Analysis       0.74      0.67      0.71       134
  Background Facts       0.61      0.70      0.65        40
        Conclusion       0.33      0.27      0.30        26
Procedural History       0.68      0.74      0.70        34
              Rule       0.75      0.82      0.78        99

          accuracy                           0.69       333
         macro avg       0.62      0.64      0.63       333
      weighted avg       0.69      0.69      0.69       333


Embedding: legal-bert
                    precision    recall  f1-score   support

          Analysis       0.80      0.75      0.77       134
  Background Facts       0.80      0.82      0.81        40
        Conclusion       0.56      0.38      0.45        26
Procedural History       0.74      0.85      0.79        34
              Rule       0.80      0.87      0.83        99

          accuracy                           0.78     

gpt-5-mini: 0it [00:00, ?it/s]


In [3]:
print(results["gpt5"])

{'report': {'Analysis': {'precision': 0.7831325301204819, 'recall': 0.4452054794520548, 'f1-score': 0.5676855895196506, 'support': 146.0}, 'Background Facts': {'precision': 0.5, 'recall': 0.875, 'f1-score': 0.6363636363636364, 'support': 40.0}, 'Conclusion': {'precision': 0.3055555555555556, 'recall': 0.8461538461538461, 'f1-score': 0.4489795918367347, 'support': 26.0}, 'Procedural History': {'precision': 0.8275862068965517, 'recall': 0.7058823529411765, 'f1-score': 0.7619047619047619, 'support': 34.0}, 'Rule': {'precision': 0.8504672897196262, 'recall': 0.7913043478260869, 'f1-score': 0.8198198198198198, 'support': 115.0}, 'accuracy': 0.6565096952908587, 'macro avg': {'precision': 0.653348316458443, 'recall': 0.7327092052746329, 'f1-score': 0.6469506798889207, 'support': 361.0}, 'weighted avg': {'precision': 0.7430012830866332, 'recall': 0.6565096952908587, 'f1-score': 0.6653577620393649, 'support': 361.0}}, 'prediction_column': 'prediction_gpt5'}


In [3]:
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

print(f"TGI GPU: {get_huggingface_llm_image_uri('huggingface')}")
print(f"TEI GPU: {get_huggingface_llm_image_uri('huggingface-tei')}")
print(f"TEI CPU: {get_huggingface_llm_image_uri('huggingface-tei-cpu')}")
print(f"TGI Neuron: {get_huggingface_llm_image_uri('huggingface-neuronx')}")

TGI GPU: 763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.6.0-tgi3.2.3-gpu-py311-cu124-ubuntu22.04
TEI GPU: 683313688378.dkr.ecr.us-east-1.amazonaws.com/tei:2.0.1-tei1.8.2-gpu-py310-cu122-ubuntu22.04
TEI CPU: 683313688378.dkr.ecr.us-east-1.amazonaws.com/tei-cpu:2.0.1-tei1.8.2-cpu-py310-ubuntu22.04
TGI Neuron: 763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.1.2-optimum0.0.28-neuronx-py310-ubuntu22.04


In [7]:
import sys
import sagemaker
import transformers
print(f"Python Version: {sys.version}")
print(f"Sagemaker Version: {sagemaker.__version__}")
print

Python Version: 3.10.12 (main, Jul  5 2023, 18:54:27) [GCC 11.2.0]
Sagemaker Version: 2.253.1
Transformer Version: 4.49.0


In [4]:
import sagemaker
print(sagemaker.__version__)

2.253.1


In [2]:
from pathlib import Path
from annotations_classification.helpers import load_annotation_spans, filter_implicit_conclusions

annotation_dir = Path("/home/lbrenap/Documents/projects/detecting_arguments/annotations/gregory")

df = load_annotation_spans(annotation_dir)
print(f"Total spans before filter: {len(df)}")
print(df["label"].value_counts(), "\n")

filtered_df = filter_implicit_conclusions(df)
print(f"Total spans after filter: {len(filtered_df)}")
print(filtered_df["label"].value_counts())


Total spans before filter: 361
label
Analysis              146
Rule                  115
Background Facts       40
Procedural History     34
Conclusion             26
Name: count, dtype: int64 

Total spans after filter: 333
label
Analysis              134
Rule                   99
Background Facts       40
Procedural History     34
Conclusion             26
Name: count, dtype: int64
